<a href="https://colab.research.google.com/github/liuchentue/SVM-based-human-detection/blob/main/Faster_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# install dependencies: 
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
Requirement already up-to-date: torch==1.5 in c:\users\admin\anaconda3\lib\site-packages (1.5.0+cu101)
Requirement already up-to-date: torchvision==0.6 in c:\users\admin\anaconda3\lib\site-packages (0.6.0+cu101)
  Using cached https://files.pythonhosted.org/packages/b6/74/c3e15707516f80192059dde32c9d78f00a88c9f3c4d1efd76cbc8d5b0a20/PyYAML-5.1-cp37-cp37m-win_amd64.whl
  Found existing installation: PyYAML 5.1.2


ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
ERROR: Invalid requirement: "'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'"
Hint: = is not a valid operator. Did you mean == ?


OSError: [WinError 126] The specified module could not be found

In [ ]:
# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2

# get image
!wget http://images.cocodataset.org/val2017/000000439715.jpg -O input.jpg
im = cv2.imread("./input.jpg")

# Create config
cfg = get_cfg()
cfg.merge_from_file("/usr/local/lib/python3.6/dist-packages/detectron2/model_zoo/configs/COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl"

# Create predictor
predictor = DefaultPredictor(cfg)

# Make prediction
outputs = predictor(im)

In [ ]:
from google.colab.patches import cv2_imshow
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])


In [ ]:
#Function for counting the number of people present in the image
def CountPerson(img):
  # Create predictor
  predictor = DefaultPredictor(cfg)
  # Make prediction
  outputs = predictor(img)
  inst=outputs["instances"]
  #extract the information from the predictor outputs
  classes=inst.get("pred_classes")
  precision=inst.get("scores")
  locations=inst.get("pred_boxes")
  #classes: what type of object is detected (for example, 0 for person; 25 for umbrella; 17 for horse) (based on COCO dataset)
  #locations: the boxes locations of the detected objects (in our case, there are 17 boxes in the example photo)
  #precision: the probability of the corresponding object classification to be accurate
  classes=tf.make_ndarray(tf.make_tensor_proto(classes.cpu())) 
  precision=tf.make_ndarray(tf.make_tensor_proto(precision.cpu())) 
  locations=tf.make_ndarray(tf.make_tensor_proto(locations.tensor.cpu()))
  numofperson=0
  for i in range(len(classes)):
    if (classes[i]==0):
      numofperson=numofperson+1
  print("Number of people in the picture:",numofperson)


#Example showing how to use this function with the sample image
CountPerson(im)

In [ ]:
 #Function for extracting the person boxes out from the image using the Faster RCNN
def Identifyperson(img, personlist): 
  # Create predictor
  predictor = DefaultPredictor(cfg)
  # Make prediction
  outputs = predictor(img)
  inst=outputs["instances"]
  #extract the information from the predictor outputs
  classes=inst.get("pred_classes")
  precision=inst.get("scores")
  locations=inst.get("pred_boxes")
  #classes: what type of object is detected (for example, 0 for person; 25 for umbrella; 17 for horse) (based on COCO dataset)
  #locations: the boxes locations of the detected objects (in our case, there are 17 boxes in the example photo)
  #precision: the probability of the corresponding object classification to be accurate
  classes=tf.make_ndarray(tf.make_tensor_proto(classes.cpu())) 
  precision=tf.make_ndarray(tf.make_tensor_proto(precision.cpu())) 
  locations=tf.make_ndarray(tf.make_tensor_proto(locations.tensor.cpu()))
  #append the boxes that are predicted to be person into the output list
  for i in range (len(classes)):
    if(classes[i]==0):
      personlist.append(locations[i])


#Example showing how to use this function with the sample image
personboxes=[]
Identifyperson(im,personboxes)
print(personboxes)

In [ ]:
#functions for cropping the person boxes as indicated by the locations and display them
import matplotlib.pyplot as plt
def plot_gallery(images, h, w, n_row=1,n_col=6):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(cv2.cvtColor(images[i],cv2.COLOR_BGR2RGB))
        print(np.shape(images[i]))
        plt.xticks(())
        plt.yticks(())

def cropboxes(testim, bx,personset):
  for i in range(len(bx)):
    bxi=bx[i]
    y1=int(bxi[0])
    y2=int(bxi[2])
    x1=int(bxi[1])
    x2=int(bxi[3])
    personi=testim[x1:x2,y1:y2]
    personset.append(personi)
 # plot_gallery(personset,100,30)  


#showing an example of cropping using the sample image
personimgs= []
cropboxes(im,personboxes,personimgs)    



In [ ]:
# to clear directory
#!rm -rf SVM-based-human-detection

# to clone directory, -b for specific branch
!git clone -b mtk3-loading-images https://github.com/liuchentue/SVM-based-human-detection.git

img0001 = cv2.imread("SVM-based-human-detection/frames/seq_000001.jpg")



# Make prediction
outputs = predictor(img0001)

v = Visualizer(img0001[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# All images written into list  images
import sys, os, glob
images = [cv2.imread(file) for file in sorted(glob.glob("SVM-based-human-detection/frames/*.jpg"))]

In [ ]:
# Part1 is finished here
for i in range (50):
  print("Image seq_",i+1)
  CountPerson(images[i])


In [ ]:
person_dataset=[]
for i in range (100):
  pb=[]
  Identifyperson(images[i],pb)
  cropboxes(images[i],pb,person_dataset)

print(len(person_dataset))
cv2_imshow(person_dataset[0])